In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
ltc_requests = requests.get("http://publicreporting.ltchomes.net/en-ca/Search_Selection.aspx")

In [3]:
ltc_requests.status_code
ltc_requests.reason

'OK'

In [4]:
print(ltc_requests.request.headers)

{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [5]:
ltc_requests.headers


{'Cache-Control': 'private', 'Content-Type': 'text/html; charset=utf-8', 'Server': 'Microsoft-IIS/8.5', 'Set-Cookie': 'ASP.NET_SessionId=udjjxslomnwgl2dbozvsqokx; path=/; HttpOnly; SameSite=Lax', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Date': 'Mon, 11 May 2020 20:18:21 GMT', 'Content-Length': '201949'}

In [6]:
ltc = BeautifulSoup(ltc_requests.text)

In [7]:
ordered_lists = ltc.find_all("ol")
ltc_list = ordered_lists[1]
home = ltc_list.find_all("a", {"class":"rsLink"})
len(home)

651

In [8]:
names = []
links = []
for each in home:
    names.append(each.get('title'))
    links.append(each.get('href'))

In [9]:
# create list with full url for each ltc home
full_links = []
for i in range(len(links)):
    full_links.append('http://publicreporting.ltchomes.net/en-ca/' + links[i])

In [39]:
# iterate through each of the links and scrape for address, city, postal code, LHIN, Administrator, Licensee, Management Firm, Home Type, Licensed Beds, Residents' Council, Family Council and Accreditation
addresses = []
cities_postalcodes = []
LHIN = []
licensee = []
management = []
home_type = []
beds = []
short_stay = []
residents_council = []
family_council = []
accreditation = []

for each in full_links:
    soup = requests.get(each)
    soup = BeautifulSoup(soup.text)
    addresses.append(soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeAddress"}).getText())
    cities_postalcodes.append(soup.find("div", {'id':'ctl00_ContentPlaceHolder1_divHomeCity'}).getText())
    profiles = soup.find("div", {'id':'ctl00_ContentPlaceHolder1_divHomeProfile_item_Col1' })
    profile_data = profiles.find_all("div", {"class":'Profilerow_col2'})
    
    try:
        LHIN.append(profile_data[0].getText())
        licensee.append(profile_data[3].getText())
        management.append(profile_data[4].getText())
        home_type.append(profile_data[5].getText())
        beds.append(profile_data[6].getText())
        short_stay.append(profile_data[7].getText())
        residents_council.append(profile_data[8].getText())
        family_council.append(profile_data[9].getText())
        accreditation.append(profile_data[10].getText())
    
    except :
        pass
        
    

In [40]:
len(LHIN)

649

In [43]:
import pandas as pd
df =[]

df = pd.DataFrame(df, columns = ['Adress', "City_PostalCode", "LHIN", "licensee","management" 
                                 
                                 
                                 
                                 
                                 
        , 'home_type', 'beds', 'short_stay', 'residents_council', 'family_council', 'accreditation'])

In [44]:
df

,Adress,City_PostalCode,LHIN,licensee,management,home_type,beds,short_stay,residents_council,family_council,accreditation


In [52]:
df = pd.DataFrame({'Address':addresses, "City, PostalCode":cities_postalcodes, 'LHIN':LHIN, 'licensee': licensee, 'management':management, 'home type':home_type, 'beds':beds, 'short stay':short_stay, 'residents counctil':residents_council, 'family council':family_council, ''})
            


ValueError: arrays must all be same length

In [53]:
df

,Address,City_PostalCode
0,1200 Afton Drive,"Sarnia, N7S6L6"
1,5050 Hillside Drive,"Beamsville, L0R1B2"
2,329 Parkside Drive P. O. Box 50,"Waterdown, L0R2H0"
3,135 Dawson Street,"Thessalon, P0R1L0"
4,145 Dawson Street,"Thessalon, P0R1L0"
5,207 Turcotte Park Road P.O. Box 270,"Mattawa, P0H1V0"
6,185 Ontario Street South,"Milton, L9T2M4"
7,333 Country Street P.O. Box 250,"Almonte, K0A1A0"
8,92 Island Road,"Scarborough, M1C2P5"
9,85 Main Street North,"Hagersville, N0A1H0"
